In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import math
from scipy.stats import norm
from scipy.stats import sem

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="white")
from openpyxl import load_workbook

In [3]:
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.api as sm

# load raw data

In [4]:
import glob
import os

In [5]:
os.chdir('...\\raw data')  #replace the "..." to a full directory the abet ii raw file

In [6]:
df_list=glob.glob('*.csv')

In [7]:
df=pd.DataFrame()

# combine ABET II export chunk

In [8]:
for i in df_list:
    df_=pd.read_csv(i)
    print(df_.shape)
    df=pd.concat([df, df_], ignore_index=True)
    print(df.shape)

(124, 132)
(124, 132)
(124, 143)
(248, 143)
(96, 132)
(344, 143)
(121, 132)
(465, 143)


In [9]:
df.loc[:, "Schedule run date short"]=pd.to_datetime(df["Schedule run date"]).dt.date

In [10]:
df["Schedule run date short"].unique()

array([datetime.date(2022, 4, 1), datetime.date(2022, 3, 31),
       datetime.date(2022, 3, 30), datetime.date(2022, 3, 29),
       datetime.date(2022, 3, 28), datetime.date(2022, 4, 8),
       datetime.date(2022, 4, 7), datetime.date(2022, 4, 6),
       datetime.date(2022, 4, 5), datetime.date(2022, 4, 4),
       datetime.date(2022, 4, 14), datetime.date(2022, 4, 13),
       datetime.date(2022, 4, 12), datetime.date(2022, 4, 11),
       datetime.date(2022, 4, 22), datetime.date(2022, 4, 21),
       datetime.date(2022, 4, 20), datetime.date(2022, 4, 19),
       datetime.date(2022, 4, 18)], dtype=object)

In [11]:
df.head(2)

,Schedule run date,Schedule name,Environment name,Group ID,Animal ID,Correct Choice Latency Bin last 45min,Incorrect Choice Latency Bin last 45min,Reward Retrieval Latency Bin last 45min,Correct Choice Latency Bin first 45min,Incorrect Choice Latency Bin first 45min,...,15 min bin - Centre ITI Touches (7),15 min bin - Right ITI Touches (7),15 min bin - Blank Touches while image shown (7),15 min bin - Hits (7),15 min bin - Misses (7),15 min bin - Mistakes (7),15 min bin - Correct Rejections (7),15 min bin - Correction Trial Correct Rejections (7),15 min bin - Correction Trial Mistakes (7),Schedule run date short
0,4/1/2022 11:04:32 AM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL,Chamber4 [4],B,R-8,0.0,0.0,0.0,1.587,1.304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-01
1,4/1/2022 11:04:29 AM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL,Chamber3 [3],B,L-7,0.0,0.0,0.0,1.548,2.034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-01


add key by using id and run time

In [12]:
df.loc[:, "key"]=df['Group ID'] + "-" + df["Animal ID"] + "-" + df["Schedule run date short"].astype('str')

In [13]:
df.head(2)

,Schedule run date,Schedule name,Environment name,Group ID,Animal ID,Correct Choice Latency Bin last 45min,Incorrect Choice Latency Bin last 45min,Reward Retrieval Latency Bin last 45min,Correct Choice Latency Bin first 45min,Incorrect Choice Latency Bin first 45min,...,15 min bin - Right ITI Touches (7),15 min bin - Blank Touches while image shown (7),15 min bin - Hits (7),15 min bin - Misses (7),15 min bin - Mistakes (7),15 min bin - Correct Rejections (7),15 min bin - Correction Trial Correct Rejections (7),15 min bin - Correction Trial Mistakes (7),Schedule run date short,key
0,4/1/2022 11:04:32 AM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL,Chamber4 [4],B,R-8,0.0,0.0,0.0,1.587,1.304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-01,B-R-8-2022-04-01
1,4/1/2022 11:04:29 AM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL,Chamber3 [3],B,L-7,0.0,0.0,0.0,1.548,2.034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-01,B-L-7-2022-04-01


In [14]:
df.shape

(465, 145)

In [15]:
df.columns.to_list()

['Schedule run date',
 'Schedule name',
 'Environment name',
 'Group ID',
 'Animal ID',
 'Correct Choice Latency Bin last 45min',
 'Incorrect Choice Latency Bin last 45min',
 'Reward Retrieval Latency Bin last 45min',
 'Correct Choice Latency Bin first 45min',
 'Incorrect Choice Latency Bin first 45min',
 'Reward Retrieval Latency Bin first 45min',
 'Correct Choice Latency Bin first 45min_sd',
 'Incorrect Choice Latency Bin first 45min_sd',
 'Reward Retrieval Latency Bin first 45min_sd',
 'Correct Choice Latency Bin last 45min_sd',
 'Incorrect Choice Latency Bin last 45min_sd',
 'Reward Retrieval Latency Bin last 45min_sd',
 '15 min bin - No of non correction trials (1)',
 '15 min bin - No of non correction trials (2)',
 '15 min bin - No of non correction trials (3)',
 '15 min bin - No of non correction trials (4)',
 '15 min bin - No of non correction trials (5)',
 '15 min bin - No of non correction trials (6)',
 '15 min bin - Left ITI Touches (1)',
 '15 min bin - Left ITI Touches (2)'

drop rows with number of hits is 0

In [16]:
df_clean=df.drop(df.loc[df['End Summary - Hits (1)']==0].index)

In [17]:
df_clean.shape

(434, 145)

find the duplicated rows

In [18]:
t=df_clean[df_clean.duplicated('key', keep=False)].sort_values(by='key')

In [19]:
t

,Schedule run date,Schedule name,Environment name,Group ID,Animal ID,Correct Choice Latency Bin last 45min,Incorrect Choice Latency Bin last 45min,Reward Retrieval Latency Bin last 45min,Correct Choice Latency Bin first 45min,Incorrect Choice Latency Bin first 45min,...,15 min bin - Right ITI Touches (7),15 min bin - Blank Touches while image shown (7),15 min bin - Hits (7),15 min bin - Misses (7),15 min bin - Mistakes (7),15 min bin - Correct Rejections (7),15 min bin - Correction Trial Correct Rejections (7),15 min bin - Correction Trial Mistakes (7),Schedule run date short,key
9,3/31/2022 12:07:12 PM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL TOT,Chamber3 [3],B,L-7,1.832,1.507,2.080,1.432,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-L-7-2022-03-31
13,3/31/2022 12:03:42 PM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL TOT,Chamber3 [3],B,L-7,0.000,0.000,0.000,1.672,2.362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-L-7-2022-03-31
11,3/31/2022 12:07:06 PM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL TOT,Chamber1 [1],B,N-5,1.304,1.305,1.508,1.396,1.350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-N-5-2022-03-31
15,3/31/2022 12:03:37 PM,Mouse ICPT Stage 3 Var ITI v1 HORIZONTAL TOT,Chamber1 [1],B,N-5,0.000,0.000,0.000,0.998,1.389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-N-5-2022-03-31
8,3/31/2022 12:07:14 PM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL TOT,Chamber4 [4],B,R-8,1.621,1.436,1.011,1.320,1.446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-R-8-2022-03-31
12,3/31/2022 12:03:45 PM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL TOT,Chamber4 [4],B,R-8,0.000,0.000,0.000,1.028,0.344,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-R-8-2022-03-31
10,3/31/2022 12:07:09 PM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL TOT,Chamber2 [2],B,RL-6,1.459,0.906,1.458,1.539,1.237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-RL-6-2022-03-31
14,3/31/2022 12:03:40 PM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL TOT,Chamber2 [2],B,RL-6,0.000,0.000,0.000,1.091,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,B-RL-6-2022-03-31
387,4/22/2022 11:13:01 AM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL,Chamber2 [2],Y,L-10,0.000,0.000,0.000,1.272,1.201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-22,Y-L-10-2022-04-22
384,4/22/2022 12:02:05 PM,Mouse ICPT Stage 3 Var ITI v1 VERTICAL,Chamber2 [2],Y,L-10,0.000,0.000,0.000,2.317,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-22,Y-L-10-2022-04-22


write down duplicated rows index number into list

In [20]:
drop_list=[13, 15, 12, 14, 391, 384, 389, 392, 390]

drop the duplicated rows with 0 number in Correct Choice Latency Bin last 45min. 
W cage mice runs under Y cage mice schedule at 4/22/2022, delted the Y group data at eariler time, since that is actually for W cage mice

In [21]:
df_clean=df_clean.drop(drop_list)
df_clean.shape

(425, 145)

# Add dosage feature to df

In [23]:
path="..\\drug schedule.xlsx"  #replace the "..." to a full directory point to the drug schedule file 
orignial_ws=load_workbook(path)
print (orignial_ws.sheetnames)

['Sheet1', 'Sheet3', 'Sheet2']


In [24]:
drug_key=pd.read_excel(path, sheet_name='Sheet1', index_col=None, header=0)

In [25]:
drug_key=drug_key.melt(id_vars=['group id','animal id','gender'], var_name="run_dt", value_name="dosage")

In [26]:
drug_key

,group id,animal id,gender,run_dt,dosage
0,W,R-1,male,2022-03-14,DS-baseline
1,W,L-2,male,2022-03-14,baseline
2,W,RL-3,male,2022-03-14,baseline
3,W,N-4,male,2022-03-14,X
4,X,N-5,male,2022-03-14,baseline
...,...,...,...,...,...
1291,A,N-4,female,2022-05-27,done
1292,B,N-5,female,2022-05-27,done
1293,B,RL-6,female,2022-05-27,done
1294,B,L-7,female,2022-05-27,done


In [27]:
drug_key.loc[:, "key"]=drug_key['group id'].astype("str") + "-" + drug_key["animal id"].astype("str") + "-" + drug_key["run_dt"].astype('str')

In [28]:
drug_key.head()

,group id,animal id,gender,run_dt,dosage,key
0,W,R-1,male,2022-03-14,DS-baseline,W-R-1-2022-03-14
1,W,L-2,male,2022-03-14,baseline,W-L-2-2022-03-14
2,W,RL-3,male,2022-03-14,baseline,W-RL-3-2022-03-14
3,W,N-4,male,2022-03-14,X,W-N-4-2022-03-14
4,X,N-5,male,2022-03-14,baseline,X-N-5-2022-03-14


# merge df and drug_key

In [29]:
df_amph=df_clean.merge(drug_key, on="key")

In [30]:
t=df_amph[df_amph.duplicated('key', keep=False)].sort_values(by='key')
t

,Schedule run date,Schedule name,Environment name,Group ID,Animal ID,Correct Choice Latency Bin last 45min,Incorrect Choice Latency Bin last 45min,Reward Retrieval Latency Bin last 45min,Correct Choice Latency Bin first 45min,Incorrect Choice Latency Bin first 45min,...,15 min bin - Correct Rejections (7),15 min bin - Correction Trial Correct Rejections (7),15 min bin - Correction Trial Mistakes (7),Schedule run date short,key,group id,animal id,gender,run_dt,dosage


hand clean on the saved csv file, and then save it

In [32]:
df_amph.groupby(by=["key", "dosage"])['Schedule run date'].count().unique()

array([1], dtype=int64)

# data clean up is done, some drug dosage point is missed. Add extra feature column in below

add gander

In [33]:
df_amph.loc[:, "gander"]=np.where(df_amph['Group ID'].isin(["W", "X", "Y", "Z"]), "male", "female")

add impuls for total

In [34]:
df_amph.loc[:, 'total-impuls'] = df_amph.loc[:, 'End Summary - Centre ITI Touches (1)'] / (df_amph.loc[:, 'End Summary - Centre ITI Touches (1)'] + df_amph.loc[:, 'End Summary - Left ITI Touches (1)'] + df_amph.loc[:, 'End Summary - Right ITI Touches (1)'])

calculate parameter for 15 min bins

In [35]:
def hr_calcu(hit, miss):
    hr_rate = hit/(hit+miss)
    return hr_rate

def fr_calcu(mistake, correct_rejection):
    fr_rate = mistake/(mistake+correct_rejection)
    return fr_rate

def d_calcu(hr,fr):
    d=norm.ppf(hr)-norm.ppf(fr)
    return d  

def si_calcu(hr,fr):
    si=(hr-fr)/(2*(hr+fr)-pow((hr+fr), 2))
    return si

def c_calcu(hr,fr):
    c=-(norm.ppf(hr)+norm.ppf(fr))/2
    return c

def ri_calcu(hr,fr):
    ri=(hr+fr-1)/(1-pow((hr-fr), 2))
    return ri

def rp_calcu(centre_ITI_Touches, hit, miss, mistake, correct_rejection, Correction_Trial_Correct_Rejection, Correction_Trial_Mistakes):
    rp=100*(centre_ITI_Touches)/(hit+miss+mistake+correct_rejection+Correction_Trial_Correct_Rejection+Correction_Trial_Mistakes)
    return rp

In [36]:
df_amph.columns.tolist()

['Schedule run date',
 'Schedule name',
 'Environment name',
 'Group ID',
 'Animal ID',
 'Correct Choice Latency Bin last 45min',
 'Incorrect Choice Latency Bin last 45min',
 'Reward Retrieval Latency Bin last 45min',
 'Correct Choice Latency Bin first 45min',
 'Incorrect Choice Latency Bin first 45min',
 'Reward Retrieval Latency Bin first 45min',
 'Correct Choice Latency Bin first 45min_sd',
 'Incorrect Choice Latency Bin first 45min_sd',
 'Reward Retrieval Latency Bin first 45min_sd',
 'Correct Choice Latency Bin last 45min_sd',
 'Incorrect Choice Latency Bin last 45min_sd',
 'Reward Retrieval Latency Bin last 45min_sd',
 '15 min bin - No of non correction trials (1)',
 '15 min bin - No of non correction trials (2)',
 '15 min bin - No of non correction trials (3)',
 '15 min bin - No of non correction trials (4)',
 '15 min bin - No of non correction trials (5)',
 '15 min bin - No of non correction trials (6)',
 '15 min bin - Left ITI Touches (1)',
 '15 min bin - Left ITI Touches (2)'

In [37]:
# #another way to replace "space" in the column name
column_name_list = df_amph.columns.tolist()
new_column=[]
for i in column_name_list:
    i_=i.replace(' ','_')
    new_column.append(i_)

df_amph.columns=new_column
# first_20_session['Environment_name']

In [38]:
df_amph.columns.tolist()

['Schedule_run_date',
 'Schedule_name',
 'Environment_name',
 'Group_ID',
 'Animal_ID',
 'Correct_Choice_Latency_Bin_last_45min',
 'Incorrect_Choice_Latency_Bin_last_45min',
 'Reward_Retrieval_Latency_Bin_last_45min',
 'Correct_Choice_Latency_Bin_first_45min',
 'Incorrect_Choice_Latency_Bin_first_45min',
 'Reward_Retrieval_Latency_Bin_first_45min',
 'Correct_Choice_Latency_Bin_first_45min_sd',
 'Incorrect_Choice_Latency_Bin_first_45min_sd',
 'Reward_Retrieval_Latency_Bin_first_45min_sd',
 'Correct_Choice_Latency_Bin_last_45min_sd',
 'Incorrect_Choice_Latency_Bin_last_45min_sd',
 'Reward_Retrieval_Latency_Bin_last_45min_sd',
 '15_min_bin_-_No_of_non_correction_trials_(1)',
 '15_min_bin_-_No_of_non_correction_trials_(2)',
 '15_min_bin_-_No_of_non_correction_trials_(3)',
 '15_min_bin_-_No_of_non_correction_trials_(4)',
 '15_min_bin_-_No_of_non_correction_trials_(5)',
 '15_min_bin_-_No_of_non_correction_trials_(6)',
 '15_min_bin_-_Left_ITI_Touches_(1)',
 '15_min_bin_-_Left_ITI_Touches_(2)'

In [39]:
df_amph.rename(columns={'Correct_Choice_Latency_Bin_1_(1-15min)': 'corr_latency_bin_1',
 'Correct_Choice_Latency_Bin_2_(15-30min)': 'corr_latency_bin_2',
 'Correct_Choice_Latency_Bin_3_(30-45min)': 'corr_latency_bin_3',
 'Correct_Choice_Latency_Bin_4_(45-60min)': 'corr_latency_bin_4',
 'Correct_Choice_Latency_Bin_5_(60-75min)': 'corr_latency_bin_5',
 'Correct_Choice_Latency_Bin_6_(75-90min)': 'corr_latency_bin_6',
 'Incorrect_Choice_Latency_Bin_1_(1-15min)': 'incorr_latency_bin_1',
 'Incorrect_Choice_Latency_Bin_2_(15-30min)': 'incorr_latency_bin_2',
 'Incorrect_Choice_Latency_Bin_3_(30-45min)': 'incorr_latency_bin_3',
 'Incorrect_Choice_Latency_Bin_4_(45-60min)': 'incorr_latency_bin_4',
 'Incorrect_Choice_Latency_Bin_5_(60-75min)': 'incorr_latency_bin_5',
 'Incorrect_Choice_Latency_Bin_6_(75-90min)': 'incorr_latency_bin_6',
 'Reward_Retrieval_Latency_Bin_1_(0-15min)':'reward_latency_bin_1',
 'Reward_Retrieval_Latency_Bin_2_(15-30min)':'reward_latency_bin_2',
 'Reward_Retrieval_Latency_Bin_3_(30-45min)':'reward_latency_bin_3',
 'Reward_Retrieval_Latency_Bin_4_(45-60min)':'reward_latency_bin_4',
 'Reward_Retrieval_Latency_Bin_5_(60-75min)':'reward_latency_bin_5',
 'Reward_Retrieval_Latency_Bin_6_(75-90min)':'reward_latency_bin_6',
 'Correct_Choice_Latency_Bin_1_(1-15min)_sd': 'corr_latency_bin_1_sd',
 'Correct_Choice_Latency_Bin_2_(15-30min)_sd': 'corr_latency_bin_2_sd',
 'Correct_Choice_Latency_Bin_3_(30-45min)_sd': 'corr_latency_bin_3_sd',
 'Correct_Choice_Latency_Bin_4_(45-60min)_sd': 'corr_latency_bin_4_sd',
 'Correct_Choice_Latency_Bin_5_(60-75min)_sd': 'corr_latency_bin_5_sd',
 'Correct_Choice_Latency_Bin_6_(75-90min)_sd': 'corr_latency_bin_6_sd',
 'Incorrect_Choice_Latency_Bin_1_(1-15min)_sd': 'incorr_latency_bin_1_sd',
 'Incorrect_Choice_Latency_Bin_2_(15-30min)_sd': 'incorr_latency_bin_2_sd',
 'Incorrect_Choice_Latency_Bin_3_(30-45min)_sd': 'incorr_latency_bin_3_sd',
 'Incorrect_Choice_Latency_Bin_4_(45-60min)_sd': 'incorr_latency_bin_4_sd',
 'Incorrect_Choice_Latency_Bin_5_(60-75min)_sd': 'incorr_latency_bin_5_sd',
 'Incorrect_Choice_Latency_Bin_6_(75-90min)_sd': 'incorr_latency_bin_6_sd',
 'Reward_Retrieval_Latency_Bin_1_(0-15min)_sd': 'reward_latency_bin_1_sd',
 'Reward_Retrieval_Latency_Bin_2_(15-30min)_sd': 'reward_latency_bin_2_sd',
 'Reward_Retrieval_Latency_Bin_3_(30-45min)_sd': 'reward_latency_bin_3_sd',
 'Reward_Retrieval_Latency_Bin_4_(45-60min)_sd': 'reward_latency_bin_4_sd',
 'Reward_Retrieval_Latency_Bin_5_(60-75min)_sd': 'reward_latency_bin_5_sd',
 'Reward_Retrieval_Latency_Bin_6_(75-90min)_sd': 'reward_latency_bin_6_sd'}, inplace=True)

In [40]:
# repalce the value 0 in the mistake to be 1, in order to calculate d'
# cols=['15_min_bin_-_Mistakes_(1)',
#  '15_min_bin_-_Mistakes_(2)',
#  '15_min_bin_-_Mistakes_(3)',
#  '15_min_bin_-_Mistakes_(4)',
#  '15_min_bin_-_Mistakes_(5)',
#  '15_min_bin_-_Mistakes_(6)']

# cols_1=['15_min_bin_-_Mistakes_(1)',
#  '15_min_bin_-_Mistakes_(2)',
#  '15_min_bin_-_Mistakes_(3)',
#  '15_min_bin_-_Mistakes_(4)',
#  '15_min_bin_-_Mistakes_(5)',
#  '15_min_bin_-_Mistakes_(6)', 
# '15_min_bin_-_Hits_(1)',
#  '15_min_bin_-_Hits_(2)',
#  '15_min_bin_-_Hits_(3)',
#  '15_min_bin_-_Hits_(4)',
#  '15_min_bin_-_Hits_(5)',
#  '15_min_bin_-_Hits_(6)']

# #df_amph[cols]=df_amph[cols].replace(to_replace=0, value=1)

# df_amph[cols]=df_amph[cols].replace(to_replace=0, value=1)

# Add total d related column

In [41]:
df_amph.loc[:, 'total-hr']=hr_calcu(df_amph.loc[:,'End_Summary_-_Hits_(1)' ], df_amph.loc[:, 'End_Summary_-_Misses_(1)'])
df_amph.loc[:, 'total-fr']=fr_calcu(df_amph.loc[:,'End_Summary_-_Mistakes_(1)' ], df_amph.loc[:, 'End_Summary_-_Correct_Rejections_(1)'])
df_amph.loc[:, 'total-d']=d_calcu(df_amph.loc[:, 'total-hr'], df_amph.loc[:, 'total-fr'])
df_amph.loc[:, 'total-si']=si_calcu(df_amph.loc[:, 'total-hr'], df_amph.loc[:, 'total-fr'])
df_amph.loc[:, 'total-c']=c_calcu(df_amph.loc[:, 'total-hr'], df_amph.loc[:, 'total-fr'])
df_amph.loc[:, 'total-ri']=ri_calcu(df_amph.loc[:, 'total-hr'], df_amph.loc[:, 'total-fr'])

# Add 15 min and 15 min bin column

In [42]:
df_amph.loc[:,'End_Summary_-_Correction_Trial_Correct_Rejection_(1)' ]=df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(1)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(2)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(3)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(4)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(5)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Correct_Rejections_(6)']
df_amph.loc[:,'End_Summary_-_Correction_Trial_Mistakes_(1)' ]=df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(1)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(2)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(3)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(4)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(5)']+df_amph.loc[:, '15_min_bin_-_Correction_Trial_Mistakes_(6)']

In [44]:
#generate new column for hr, fr, d, si, c, ri, rp
from string import Template
#loop 6 different time bin
df_list=[df_amph]

df=df_amph
bin_number=["1", "2", "3", "4", "5", "6"]
for i in bin_number:

    #local 3 different time bin column
    hit='15_min_bin_-_Hits_(%s)'%(i)
    hit_c=df_amph.loc[:, hit]

    miss = '15_min_bin_-_Misses_(%s)'%(i)
    miss_c=df_amph.loc[:,miss]

    mistake = '15_min_bin_-_Mistakes_(%s)'%(i)
    mistake_c=df_amph.loc[:,mistake]

    correct_rejection='15_min_bin_-_Correct_Rejections_(%s)'%(i)
    correct_rejection_c=df_amph.loc[:,correct_rejection]

    center_iti_touches = '15_min_bin_-_Centre_ITI_Touches_(%s)'%(i)
    center_iti_touches_c=df_amph.loc[:,center_iti_touches]

    Correction_Trial_Correct_Rejection = '15_min_bin_-_Correction_Trial_Correct_Rejections_(%s)'%(i)
    Correction_Trial_Correct_Rejection_c=df_amph.loc[:,Correction_Trial_Correct_Rejection]

    Correction_Trial_Mistakes = '15_min_bin_-_Correction_Trial_Mistakes_(%s)'%(i)
    Correction_Trial_Mistakes_c = df_amph.loc[:,Correction_Trial_Mistakes]

    #calculat hr and fr first
    df_amph.loc[:,'hr_15min_bin_%s'%i]=hr_calcu(hit_c,miss_c)
    df_amph.loc[:,'fr_15min_bin_%s'%i]=fr_calcu(mistake_c, correct_rejection_c)
df_amph.to_csv(r"D:\CPT_TOT_manuscript\data processing\amph_withzero_hr_fr.csv")  

df_zero = df[(df["hr_15min_bin_1"] == 0) |
                   (df["hr_15min_bin_2"] == 0) |
                   (df["hr_15min_bin_3"] == 0) |
                   (df["hr_15min_bin_4"] == 0) |
                   (df["hr_15min_bin_5"] == 0) |
                   (df["hr_15min_bin_6"] == 0) |
                   (df['fr_15min_bin_1'] == 0) |
                   (df['fr_15min_bin_2'] == 0) |
                   (df['fr_15min_bin_3'] == 0) |
                   (df['fr_15min_bin_4'] == 0) |
                   (df['fr_15min_bin_5'] == 0) |
                   (df['fr_15min_bin_6'] == 0)]

df_zero.to_csv("......", index=False)  #replace the "......" to a full directory you want to save the file 

for i in bin_number:
     #local 3 different time bin column
    hit='15_min_bin_-_Hits_(%s)'%(i)
    hit_c=df_amph.loc[:, hit]

    miss = '15_min_bin_-_Misses_(%s)'%(i)
    miss_c=df_amph.loc[:,miss]

    mistake = '15_min_bin_-_Mistakes_(%s)'%(i)
    mistake_c=df_amph.loc[:,mistake]

    correct_rejection='15_min_bin_-_Correct_Rejections_(%s)'%(i)
    correct_rejection_c=df_amph.loc[:,correct_rejection]

    center_iti_touches = '15_min_bin_-_Centre_ITI_Touches_(%s)'%(i)
    center_iti_touches_c=df_amph.loc[:,center_iti_touches]

    Correction_Trial_Correct_Rejection = '15_min_bin_-_Correction_Trial_Correct_Rejections_(%s)'%(i)
    Correction_Trial_Correct_Rejection_c=df_amph.loc[:,Correction_Trial_Correct_Rejection]

    Correction_Trial_Mistakes = '15_min_bin_-_Correction_Trial_Mistakes_(%s)'%(i)
    Correction_Trial_Mistakes_c = df_amph.loc[:,Correction_Trial_Mistakes]
        
    #check if hr or fr is value 0, than replace the zero with p concept.
    for e in range(len(df)): 
        if df_amph.iloc[e]['hr_15min_bin_%s'%i]==0:
            #display(df_amph.loc[[e]])
            df_amph.at[e, 'hr_15min_bin_%s'%i]=1/(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection])
        if df_amph.iloc[e]['hr_15min_bin_%s'%i]==1:
            #display(df_amph.loc[[e]])
            df_amph.at[e, 'hr_15min_bin_%s'%i]=(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection]-1)/(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection])        
        if df_amph.iloc[e]['fr_15min_bin_%s'%i]==0:
            #display(df_amph.loc[[e]])
            df_amph.at[e, 'fr_15min_bin_%s'%i]=1/(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection])
        if df_amph.iloc[e]['fr_15min_bin_%s'%i]==1:
            #display(df_amph.loc[[e]])
            df_amph.at[e, 'fr_15min_bin_%s'%i]=(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection]-1)/(df_amph.iloc[e][hit]+df_amph.iloc[e][miss]+df_amph.iloc[e][mistake]+df_amph.iloc[e][correct_rejection]) 
        
    df_amph.loc[:,'d_15min_bin_%s'%i]=d_calcu(df_amph.loc[:,'hr_15min_bin_%s'%i], df_amph.loc[:,'fr_15min_bin_%s'%i])
    df_amph.loc[:,'si_15min_bin_%s'%i]=si_calcu(df_amph.loc[:,'hr_15min_bin_%s'%i], df_amph.loc[:,'fr_15min_bin_%s'%i])
    df_amph.loc[:,'c_15min_bin_%s'%i]=c_calcu(df_amph.loc[:,'hr_15min_bin_%s'%i], df_amph.loc[:,'fr_15min_bin_%s'%i])
    df_amph.loc[:,'ri_15min_bin_%s'%i]=ri_calcu(df_amph.loc[:,'hr_15min_bin_%s'%i], df_amph.loc[:,'fr_15min_bin_%s'%i])
    df_amph.loc[:,'rp_15min_bin_%s'%i]=rp_calcu(center_iti_touches_c, hit_c, miss_c, mistake_c, correct_rejection_c, 
                                               Correction_Trial_Correct_Rejection_c, Correction_Trial_Mistakes_c)

df_amph.loc[:, 'Hit_Rate'] =hr_calcu(df_amph.loc[:, 'End_Summary_-_Hits_(1)'],df_amph.loc[:, 'End_Summary_-_Misses_(1)'])
df_amph.loc[:, 'False_Alarm_Rate'] =fr_calcu(df_amph.loc[:, 'End_Summary_-_Mistakes_(1)'], df_amph.loc[:, 'End_Summary_-_Correct_Rejections_(1)'])
for e in range(len(df)): 
    
    if df_amph.iloc[e]['Hit_Rate']==0:
        trial_n=df_amph.iloc[e]['End_Summary_-_Hits_(1)']+df_amph.iloc[e]['End_Summary_-_Misses_(1)']+df_amph.iloc[e]['End_Summary_-_Mistakes_(1)']+ df_amph.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df_amph.at[e, 'Hit_Rate']=1/(trial_n)
    if df_amph.iloc[e]['Hit_Rate']==1:
        trial_n=df_amph.iloc[e]['End_Summary_-_Hits_(1)']+df_amph.iloc[e]['End_Summary_-_Misses_(1)']+df_amph.iloc[e]['End_Summary_-_Mistakes_(1)']+ df_amph.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df_amph.at[e, 'Hit_Rate']=(trial_n-1)/(trial_n)
    
    if df_amph.iloc[e]['False_Alarm_Rate']==0:
        trial_n=df_amph.iloc[e]['End_Summary_-_Hits_(1)']+df_amph.iloc[e]['End_Summary_-_Misses_(1)']+df_amph.iloc[e]['End_Summary_-_Mistakes_(1)']+ df_amph.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df_amph.at[e, 'False_Alarm_Rate']=1/(trial_n)
    if df_amph.iloc[e]['False_Alarm_Rate']==1:
        trial_n=df_amph.iloc[e]['End_Summary_-_Hits_(1)']+df_amph.iloc[e]['End_Summary_-_Misses_(1)']+df_amph.iloc[e]['End_Summary_-_Mistakes_(1)']+ df_amph.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        print(str(e))
        df_amph.at[e, 'False_Alarm_Rate']=(trial_n-1)/(trial_n)
        
df_amph.loc[:, 'c'] =c_calcu(df_amph.loc[:,'Hit_Rate'], df_amph.loc[:,'False_Alarm_Rate'])
df_amph.loc[:, 'd'] =d_calcu(df_amph.loc[:,'Hit_Rate'], df_amph.loc[:,'False_Alarm_Rate'])
df_amph.loc[:, 'si'] =si_calcu(df_amph.loc[:,'Hit_Rate'], df_amph.loc[:,'False_Alarm_Rate'])
df_amph.loc[:, 'ri'] =ri_calcu(df_amph.loc[:,'Hit_Rate'], df_amph.loc[:,'False_Alarm_Rate'])
df_amph.loc[:, 'impuls'] =rp_calcu(df_amph.loc[:,'End_Summary_-_Centre_ITI_Touches_(1)'], df_amph.loc[:,'End_Summary_-_Hits_(1)' ], 
                              df_amph.loc[:, 'End_Summary_-_Misses_(1)'],df_amph.loc[:,'End_Summary_-_Mistakes_(1)'], 
                              df_amph.loc[:, 'End_Summary_-_Correct_Rejections_(1)'], df_amph.loc[:,'End_Summary_-_Correction_Trial_Correct_Rejection_(1)'], 
                              df_amph.loc[:,'End_Summary_-_Correction_Trial_Mistakes_(1)'])

In [45]:
#check if inf in d' exist
df_inf=df_amph[['d_15min_bin_1', 'd_15min_bin_2','d_15min_bin_3','d_15min_bin_4','d_15min_bin_5','d_15min_bin_6']]
count = np.isinf(df_inf).values.sum()
print("It contains " + str(count) + " infinite values")

It contains 0 infinite values


# add session order

In [46]:
df_amph['group_animal_id']=df_amph["group_id"] + "-" + df_amph["animal_id"]

In [47]:
#add session order
df_amph['session_order']=df_amph.groupby(by=['group_animal_id', 'dosage'])['Schedule_run_date_short'].transform(lambda x: x.rank())

save the data before plot

In [49]:
df_amph.to_csv("......", index=False )  #replace the "......" to a full directory you want to save the file 

# dataframe melt transfor

In [51]:
# #dateframe melt
# m_probe_15bin=m_probe_15bin.melt(id_vars=['dosage','Schedule_run_date','Schedule_name','Environment_name','Group_ID',
#  'Animal_ID','session','id','id_session','gender', 'id_dosage'], var_name="param", value_name="value")

# f_probe_15bin=f_probe_15bin.melt(id_vars=['dosage','Schedule_run_date','Schedule_name','Environment_name','Group_ID',
#  'Animal_ID','session','id','id_session','gender', 'id_dosage'], var_name="param", value_name="value")

In [52]:
#filter mask
hit_list= ['15_min_bin_-_Hits_(1)',
 '15_min_bin_-_Hits_(2)',
 '15_min_bin_-_Hits_(3)',
 '15_min_bin_-_Hits_(4)',
 '15_min_bin_-_Hits_(5)',
 '15_min_bin_-_Hits_(6)']

false_list=['15_min_bin_-_Mistakes_(1)',
 '15_min_bin_-_Mistakes_(2)',
 '15_min_bin_-_Mistakes_(3)',
 '15_min_bin_-_Mistakes_(4)',
 '15_min_bin_-_Mistakes_(5)',
 '15_min_bin_-_Mistakes_(6)']

miss_list=['15_min_bin_-_Misses_(1)',
           '15_min_bin_-_Misses_(2)',
           '15_min_bin_-_Misses_(3)',
          '15_min_bin_-_Misses_(4)',
          '15_min_bin_-_Misses_(5)',
          '15_min_bin_-_Misses_(6)']

correj_list=['15_min_bin_-_Correct_Rejections_(1)',
 '15_min_bin_-_Correct_Rejections_(2)',
 '15_min_bin_-_Correct_Rejections_(3)',
 '15_min_bin_-_Correct_Rejections_(4)',
 '15_min_bin_-_Correct_Rejections_(5)',
 '15_min_bin_-_Correct_Rejections_(6)']

hr_list = ['hr_15min_bin_1', 'hr_15min_bin_2','hr_15min_bin_3', 'hr_15min_bin_4', 'hr_15min_bin_5', 'hr_15min_bin_6']

fr_list = ['fr_15min_bin_1', 'fr_15min_bin_2', 'fr_15min_bin_3', 'fr_15min_bin_4', 'fr_15min_bin_5', 'fr_15min_bin_6']

d_list = ['d_15min_bin_1','d_15min_bin_2','d_15min_bin_3', 'd_15min_bin_4', 'd_15min_bin_5', 'd_15min_bin_6']

si_list=['si_15min_bin_1','si_15min_bin_2','si_15min_bin_3', 'si_15min_bin_4', 'si_15min_bin_5', 'si_15min_bin_6']

c_list=['c_15min_bin_1','c_15min_bin_2','c_15min_bin_3', 'c_15min_bin_4', 'c_15min_bin_5', 'c_15min_bin_6']

ri_list=['ri_15min_bin_1','ri_15min_bin_2','ri_15min_bin_3', 'ri_15min_bin_4', 'ri_15min_bin_5', 'ri_15min_bin_6']

rp_list=['rp_15min_bin_1','rp_15min_bin_2','rp_15min_bin_3', 'rp_15min_bin_4', 'rp_15min_bin_5', 'rp_15min_bin_6']

cor_latency_mean=['corr_latency_bin_1',
 'corr_latency_bin_2',
 'corr_latency_bin_3',
 'corr_latency_bin_4',
 'corr_latency_bin_5',
 'corr_latency_bin_6']

incor_latency_mean=['incorr_latency_bin_1',
 'incorr_latency_bin_2',
 'incorr_latency_bin_3',
 'incorr_latency_bin_4',
 'incorr_latency_bin_5',
 'incorr_latency_bin_6']

reward_latency_mean=['reward_latency_bin_1',
 'reward_latency_bin_2',
 'reward_latency_bin_3',
 'reward_latency_bin_4',
 'reward_latency_bin_5',
 'reward_latency_bin_6']

cor_latency_sd=['corr_latency_bin_1_sd',
 'corr_latency_bin_2_sd',
 'corr_latency_bin_3_sd',
 'corr_latency_bin_4_sd',
 'corr_latency_bin_5_sd',
 'corr_latency_bin_6_sd']

incor_latency_sd=['incorr_latency_bin_1_sd',
 'incorr_latency_bin_2_sd',
 'incorr_latency_bin_3_sd',
 'incorr_latency_bin_4_sd',
 'incorr_latency_bin_5_sd',
 'incorr_latency_bin_6_sd']

reward_latency_sd=['reward_latency_bin_1_sd',
 'reward_latency_bin_2_sd',
 'reward_latency_bin_3_sd',
 'reward_latency_bin_4_sd',
 'reward_latency_bin_5_sd',
 'reward_latency_bin_6_sd']

replace the "," in the string of dataframe to let sns convert it to float

In [53]:
for i in reward_latency_mean:
    df_amph[i] = (df_amph[i].astype(str).str.split()).apply(lambda x: float(x[0].replace(',', '')))

In [54]:
for i in ['Reward_Retrieval_Latency_Bin_first_45min', 'Reward_Retrieval_Latency_Bin_last_45min']:
    df_amph[i] = (df_amph[i].astype(str).str.split()).apply(lambda x: float(x[0].replace(',', '')))

In [55]:
#dateframe melt
df_amph_melt=df_amph.melt(id_vars=['Animal_ID','Group_ID','dosage', 'key', 'gander', 'Schedule_name', 'Environment_name', 'Schedule_run_date_short', 'Schedule_run_date', 'session_order', 'group_animal_id'], var_name="param", value_name="value")

In [57]:
t=df_amph.groupby(by=["Animal_ID", "Group_ID", "dosage"], as_index=False).count()

# check missed data point

In [59]:
k=t[t["dosage"].isin(["AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk", "AMPH-vehicle"])].groupby(by=["Animal_ID", "Group_ID"], as_index=False) .sum() .sort_values(by="Schedule_run_date").iloc[:, :4]

In [60]:
k

,Animal_ID,Group_ID,Schedule_run_date,Schedule_name
1,L-15,Z,3,3
19,RL-3,W,3,3
3,L-2,W,3,3
18,RL-14,Z,3,3
7,N-13,Z,3,3
0,L-10,Y,4,4
17,RL-11,Y,4,4
16,R-9,Y,4,4
15,R-8,X,4,4
14,R-8,B,4,4


In [61]:
k.loc[:, "missed drug session number"] = 4- k["Schedule_run_date"]

In [62]:
k

,Animal_ID,Group_ID,Schedule_run_date,Schedule_name,missed drug session number
1,L-15,Z,3,3,1
19,RL-3,W,3,3,1
3,L-2,W,3,3,1
18,RL-14,Z,3,3,1
7,N-13,Z,3,3,1
0,L-10,Y,4,4,0
17,RL-11,Y,4,4,0
16,R-9,Y,4,4,0
15,R-8,X,4,4,0
14,R-8,B,4,4,0


In [63]:
u=t[(t["Animal_ID"].isin(["L-15", "RL-3", "L-2","RL-14", "N-13"])) & (t["Group_ID"].isin(["Z", "W"])) & (t["dosage"].isin(["AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk", "AMPH-vehicle"]))]

In [64]:
pt=u.groupby(by=["Animal_ID", "Group_ID", "dosage"], as_index=False).count().iloc[:, :5]

In [65]:
pt

,Animal_ID,Group_ID,dosage,Schedule_run_date,Schedule_name
0,L-15,Z,AMPH-0.3mpk,1,1
1,L-15,Z,AMPH-1mpk,1,1
2,L-15,Z,AMPH-vehicle,1,1
3,L-2,W,AMPH-0.3mpk,1,1
4,L-2,W,AMPH-1mpk,1,1
5,L-2,W,AMPH-vehicle,1,1
6,N-13,Z,AMPH-0.3mpk,1,1
7,N-13,Z,AMPH-0.6mpk,1,1
8,N-13,Z,AMPH-1mpk,1,1
9,RL-14,Z,AMPH-0.6mpk,1,1


In [66]:
pd.pivot_table(pt, values="Schedule_run_date", index=["Group_ID", "Animal_ID"], columns=["dosage"]) #, aggfunc=np.count)

dosage              AMPH-0.3mpk  AMPH-0.6mpk  AMPH-1mpk  AMPH-vehicle
Group_ID Animal_ID                                                   
W        L-2                1.0          NaN        1.0           1.0
         RL-3               1.0          1.0        NaN           1.0
Z        L-15               1.0          NaN        1.0           1.0
         N-13               1.0          1.0        1.0           NaN
         RL-14              NaN          1.0        1.0           1.0

In [67]:
# m_probe_15bin = m_probe_15bin.replace(np.nan, '', regex=True)
# f_probe_15bin = f_probe_15bin.replace(np.nan, '', regex=True)

In [68]:
#pd.set_option("display.max_rows", 999)

In [69]:
#m_probe_15bin.param.value_counts()

# plot and stat for TOT 15 min bins

In [70]:
def normal(mean, std, histmax=False, color="black"):
    x = np.linspace(mean-4*std, mean+4*std, 200)
    p = stats.norm.pdf(x, mean, std)
    if histmax:
        p = p*histmax/max(p)
    z = plt.plot(x, p, color, linewidth=2)

In [71]:
#create path folder
save_path="........" #replace the "..." to a full directory you want to save the file 
if os.path.isdir(save_path) == False:
    os.mkdir(save_path)
        
for i in ["male", "female"]:
    df=df_amph_melt[df_amph_melt['gander'] == i]

    para_list = [d_list, si_list, ri_list, 
                 cor_latency_mean, incor_latency_mean, reward_latency_mean, 
                 cor_latency_sd, incor_latency_sd, reward_latency_sd,correj_list] 

    filename= ['tot_d_15bin', 'tot_si_15bin', 'tot_ri_15bin',
              'tot_correct_latency_15bin', 'tot_incorrect_latency_15bin', 'tot_reward_latency_15bin',
              'tot_correct_latency_sd_15bin', 'tot_incorrect_latency_sd_15bin', 'tot_reward_latency_sd_15bin', 'tot_correj_15bin']

    para_list_0 = [hit_list, false_list, miss_list,  hr_list, fr_list, c_list, rp_list]
    filename_0 = ['tot_hit_15bin', 'tot_false_15bin','tot_miss_15bin', 'tot_hit_rate_15bin', 'tot_false_alarm_rate_15bin', 'tot_c_15bin', 'tot_impuls_15bin']
    
    for j,l in zip(para_list, filename): 
        k = df[df.dosage.isin(["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk", "AMPH-1mpk"]) & df.param.isin(j)].copy()
        k["value"]=pd.to_numeric(k["value"])
        k.to_csv(save_path+"\\%s_%s.csv"%(i, l))
#         k=k.dropna()
#         k.to_csv(save_path+"\\%s_%s_dropna.csv"%(i, l))
    #     plot=sns.catplot(x='param', y='value', kind="bar", 
    #                                ci=68,capsize=.2, data=k) #order=["ds_baseline","ds"],

        plot=sns.pointplot(x='param', y='value', hue='dosage', data=k, errorbar=('ci', 68),
                            palette=["black", "green", "royalblue", "red"], 
                           hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"])
                           #capsize=.2, #palette=color_dict, hue_order=order,marker='o',legend_out=True,
        plot.set(xlabel="")
        plot.set(ylabel="")
        plt.title(i+"_"+l)
        plt.xticks(rotation = 45, ha='right')
        plt.legend(bbox_to_anchor=(1.4, 1),borderaxespad=0)
        plt.savefig(save_path+ "\\%s_%s.png"%(i, l), dpi=800, bbox_inches="tight")
        plt.close()